In [0]:
%sql
CREATE DATABASE IF NOT EXISTS db_bronze;

In [0]:
%sql
DROP TABLE IF EXISTS db_bronze.hired_employees;
CREATE TABLE db_bronze.hired_employees(
  id bigint,
  name string,
  datetime string,
  department_id bigint,
  job_id bigint
);

DROP TABLE IF EXISTS db_bronze.departments;
CREATE TABLE db_bronze.departments(
  id bigint,
  department string
);

DROP TABLE IF EXISTS db_bronze.jobs;
CREATE TABLE db_bronze.jobs(
  id bigint,
  job string
)

In [0]:
%python
# coding: utf-8

# ||********************************************************************************************************
# || PROYECTO   		: POC -CHALLENGE GLOBLANT 
# || NOMBRE     		: challenge.py
# || TABLA DESTINO	: db_bronze.hired_employees
# ||                  db_bronze.departments 
# ||                  db_bronze.jobs
# || TABLAS FUENTES	: departments.csv
# ||                  hired_employees.csv
# ||                  jobs.csv
# || OBJETIVO   		: ETL - big data migrati
# || TIPO       		: pyspark
# || REPROCESABLE	  : NA
# || SCHEDULER		  : NA
# || JOB  		      : NA
# || VERSION   DESARROLLADOR           FECHA        DESCRIPCION
# || 1.1       ALEXIS DAVILA        21/03/23     Creacion del proceso
# *************************************************************************************************************

###
 # @section Import
 ##

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

###
 # @section configuracion de recursos
 ##
spark = SparkSession.builder.\
appName("CSV to Database").\
config("spark.driver.cores","8").\
config("spark.driver.memoryOverhead","20g").\
config("spark.driver.memory", "16g").\
config("spark.executor.cores","15").\
config("spark.executor.memory","20g").\
config("spark.executor.memoryOverhead","20g").\
config("spark.dynamicAllocation.enabled","true").\
config("spark.dynamicAllocation.maxExecutors","20").\
config("spark.debug.maxToStringFields", "100").\
config("spark.shuffle.service.enabled","true").\
config("spark.sql.shuffle.partitions", "500").\
config("spark.default.parallelism", "500").\
config("spark.sql.autoBroadcastJoinThreshold","-1").\
config("spark.sql.join.preferSortMergeJoin","false").\
config("spark.maxRemoteBlockSizeFetchToMem","2147483135").\
config("spark.shuffle.spill.compress","true").\
config("spark.shuffle.compress","true").\
config("spark.ui.enabled","true").\
enableHiveSupport().\
getOrCreate()

###
 # @section funciones
 ##


def insert_csv_to_db(csv_file,table_name,nombre_columnas):
    df = spark.read.format("csv").option("header", "false").option("inferSchema","True").load(csv_file)
    nombres_columnas_final = nombre_columnas
    df = df.toDF(*nombres_columnas_final)
    df.write.format("avro").mode("append").option("batchsize", "1000").option("isolationLevel", "NONE").option("numPartitions", "10").option("mergeSchema", "true").saveAsTable(table_name)

def main():
    
    csv_file_1 = "https://raw.githubusercontent.com/alexis18daes/databricks_challenge_repo/dev/hired_employees.csv" 
    csv_file_2 = "https://raw.githubusercontent.com/alexis18daes/databricks_challenge_repo/dev/departments.csv"
    csv_file_3 = "https://raw.githubusercontent.com/alexis18daes/databricks_challenge_repo/dev/jobs.csv"
    
    table_name_1 = "db_bronze.hired_employees"
    table_name_2 = "db_bronze.departments"
    table_name_3 = "db_bronze.jobs"
    
    nombre_columnas_1 = ["id","name","datetime","department_id","job_id"]
    nombre_columnas_2 = ["id","name","department"]
    nombre_columnas_3 = ["id","job"]
    
    insert_csv_to_db(csv_file_1,table_name_1,nombre_columnas_1)
    insert_csv_to_db(csv_file_2,table_name_2,nombre_columnas_2)
    insert_csv_to_db(csv_file_3,table_name_3,nombre_columnas_3)
    
#Ejecucion
main()

spark.stop()

#Salida
exit()

